In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
data = pd.read_csv(r"C:\Users\Admin\Data python\hotel_bookings.csv")
pd.set_option('display.max_columns', 32)
data.head()

In [ ]:
col = ['agent', 'company', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'reservation_status_date', 'reservation_status', 'hotel']

hotel_book = data.drop(col, axis= 1)

In [ ]:
hotel_book.head()

In [ ]:
hotel_book.info()

In [ ]:
hotel_book.describe()

## Explain column meaning

**hotel**：City hotel hoặc Resort Hotel

**lead_time**：Số ngày từ khi đặt phòng đến ngày nghỉ trọ.

**stays_in_weekend_nights**：Số ngày đặt phòng và nghỉ lại (Thứ 7, Chủ Nhật)

**stays_in_week_nights**：Số ngày đặt phòng và nghỉ lại (Thứ 2 ~ Thứ 6)

**country**：Quốc gia.

**market_segment**：Phân loại thị trường(TA：Đại lý du lịch、TO：Điều hành du lịch)

**distribution_channel**：kênh cung cấp(TA：Đại lý du lịch、TO：Điều hành du lịch)

**is_repeated_guest**：có phải repeater hay không

**previous_cancellations**：Đã từng hủy đặt phòng trước đây

**previous_bookings_not_canceled**：chưa từng hủy đặt phòng trước đây

**reserved_room_type**：Loại phòng đặt

**assigned_room_type**：Loại phòng sắp xếp

**booking_changes**：Số lần thay đổi đặt phòng

**agent**：ẩn

**company**：ẩn

**days_in_waiting_list**：Số ngày lấy được xác nhận của khách hàng.

**customer_type**：loại khách hàng

**adr**：Tỉ giá trung bình ngày nghỉ trọ

**required_car_parking_spaces**：Yêu cầu bao nhiêu chỗ đỗ xe

**total_of_special_requests**：Số lượng yêu cầu đặc biệt.

*Đây là bộ dữ liệu trích xuất từ hệ thống Quản lý khách sạn (PMS), vì thế tên khách sạn và đại lý đặt phòng sẽ được ẩn 
đi.*

### Explain for meal

**RO**: Room only

**BB**: Bed & Breakfast

**HB**: Half Board (Breakfast and Dinner normally)

**FB**: Full Board (Beakfast, Lunch and Dinner)

**AI**: All Inclusive (all services of full board plus any others specified in each case)

In [ ]:
# Categorical data
col = ['meal', 'country',"market_segment", 'distribution_channel', 'reserved_room_type', 'assigned_room_type',"deposit_type","customer_type"]

for i in hotel_book[col]:
    x = hotel_book[i].value_counts()
    print("Column name is:",i,"and it value is:",x)
    print()

**meal country**:5

**market_segment**: 177

**market_segment**: 8 

**distribution_channel**: 5 

**reserved_room_type**: 9

**assigned_room_type**: 11

**deposit_type**: 3

**customer_type**: 4

In [ ]:
hotel_book.isnull().any()

In [ ]:
hotel_book = hotel_book.dropna()

In [ ]:
index_drop = hotel_book[(hotel_book['meal'] == 'Undefined') & (hotel_book['market_segment'] == 'Undefined') & (hotel_book['distribution_channel'] == 'Undefined')]

hotel_book.drop(index_drop.index, inplace = True,axis = 0)

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(hotel_book, test_size = 0.2, random_state = 42)

In [ ]:
hotel_book.head()

In [ ]:
X_notprepared_train_set = train_set.drop(["is_canceled"], axis = 1)

# Take the booking has been canceled
Y_train_set = (train_set["is_canceled"] == 1).copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_attri = ["lead_time", 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'booking_changes', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces']
cat_attri = ['meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type']

full_pipeline = ColumnTransformer([
    ("num", StandardScaler(), num_attri),
    ("cat", OneHotEncoder(), cat_attri)
])

X_prepared_train_set = full_pipeline.fit_transform(X_notprepared_train_set)

In [ ]:
X_prepared_train_set = X_prepared_train_set.toarray()

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.95)

X_prepared_train_set_reduced = pca.fit_transform(X_prepared_train_set)

In [ ]:
X_prepared_train_set_reduced.shape

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict

sgd_clf = SGDClassifier(random_state = 42)

sgd_clf.fit(hotel_prepared_train_set_reduced, hotel_label_train_set)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

def EvalAcc&Confumatrix(model, X, Y_pred, Y, kfold = 5):
    result1 = cross_val_score(model, X, Y, scoring = "accuracy", cv = kfold)
    result2 = cross_val_predict(model, X, Y, cv = kfold)
    return result1
    return confusion_matrix(result2, Y)

In [ ]:
from sklearn.model_selection import cross_val_predict

label_scores_pre = cross_val_predict(sgd_clf, hotel_prepared_train_set_reduced, 
                                     hotel_label_train_set, cv = 3, method = "decision_function")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(hotel_label_train_set,label_scores_pre)